# ROE选股

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年4月6日

----------
- 上市满五年，近五年每年ROE>15，近五年平均ROE>20；或者，上市满三年，近三年每年ROE>20；
- 除医药消费，其他行业只选一家（业内要么第一、要么唯一）；
- 有持续良好的经营历史及持续的分红，剔除创业板及ST；
- 近三年净利润复合增长必须为正，按近五年均ROE降序选出20家企业。

In [1]:
#导入库
import numpy as np
import pandas as pd
import time
from datetime import timedelta,date

# 聚宽数据
import jqdatasdk
jqdatasdk.auth("13695683829", "ssk741212")
from jqdatasdk import *

#引入自定义库
import sys
sys.path.append('../PF')
sys.path.append('../VD')

from pf_stk import *
from vd_stk import stkView 

roe20=Stk('csv','../Data/','roe20') 
view=stkView(roe20,u'ROE20股票')   




auth success 
数据引擎：准备好


### 1.1.生成股票池

In [4]:
#股票池筛选函数
def pool(index='000016.XSHG',stat_year=2017,max_year=5):
    #初选股票池列表
    pool=[]
    #获取沪深所有股票
    stocks=get_index_stocks(index)
    #股票总数量
    n=len(stocks)
    #计数器
    m=0
    for code in stocks:
        #计数器累加
        m=m+1
        #股票名称
        stock_name=get_security_info(code).display_name       
        #转换代码
        stock_code=code[0:6]
        #IPO日期
        ipo_date = get_security_info(code).start_date
        #转换ipo日期
        ipo_year=int(ipo_date.strftime('%Y'))
        #显示进度
        print ('\r股票筛选：%s %s %s/%s'%(stock_code,stock_name,m,n),end="")
        #剔除创业板
        if is_cyb(code):
            continue
        #剔除退市、ST
        if is_st(code):
            continue
        #剔除未满3年   
        if stat_year-ipo_year<2:
            continue    
        #判断是否已出最新年报    
        if not is_report(code,stat_year):
            continue         
        #入选标记
        isfind=False   
        #上市满5年（具有完整的最近5年年报数据）
        if stat_year-ipo_year>=4:
            #roe、毛利率、营业收入复合增长率、利润复合增长率
            (roe,roe_mean,gross_profit,inc_revenue,net_profit,new_profit,df)=get_stock_indicator(code,
                                             stat_year,max_year,5)
            #每年ROE>15,平均ROE>20
            if len(df[df.inc_return>=15.0])>=5 and roe_mean>=20:
                isfind=True
        #或者上市满3年（具有完整的最近3年年报数据）         
        elif stat_year-ipo_year>=2:
            #roe、毛利率、营业收入复合增长率、利润复合增长率
            (roe,roe_mean,gross_profit,inc_revenue,net_profit,new_profit,df)=get_stock_indicator(code,
                                               stat_year,max_year,3)
            #每年ROE>20
            df=df[0:3]
            if len(df[df.inc_return>=20.0])>=3:
                isfind=True
        #剔除利复增为负  
        if net_profit<=0.00 :
            continue  
        #剔除最新为负  
        if new_profit<=0.00 :
            continue      
        #如果入选，则添加、整理数据
        if isfind : 
            #获取roe列表
            roe_list=df['inc_return'].tolist()
            #不满5年的数据填充为NAN
            for i in range(max_year-len(df)):
                roe_list=[float('NAN')]+roe_list  
            #获取所属行业名称
            industry=get_stock_industry(code)
            #拼接列表
            pool.append([code,stock_name]+roe_list+
                             [roe_mean,gross_profit,inc_revenue,net_profit,industry,ipo_date]) 
    #整理标题头
    columns=['code','name']+['%s'%str(stat_year-max_year+1+y) for y in range(0,max_year)]+['roe_mean',
                  'gross_profit','revenue_grow','profit_grow','industry','ipo_date']
    #组织数据
    df=pd.DataFrame(data=pool,columns=columns)
    #设置索引为股票代码
    df.set_index(['code'],inplace=True)
    df.index.name=None
    #返回结果
    df=df.sort_values(['roe_mean'],ascending=False)    
    return df


#传入筛选函数
roe20.pool.create_form(pool,index='000902.XSHG',stat_year=2018,max_year=5)


股票筛选：603999 读者传媒 3682/3682

### 1.1.查看股票池

In [0]:
roe20.pool.show()

## 2.数据获取、分析
1. 设置当前股票池，以下数据获取和分析皆是针对当前股票池。
1. 获取行业列表，用于对股票进行行业分类；
2. 获取股票日线数据，包括行情数据、估值数据。

- 估值分析：PE-TTM（滚动市盈率）、PB（平均市净率）；
- 财务指标分析：毛利率、ROE（年报、扣非后净资产收益率）、近五年或三年的平均ROE、近三年营业收入复合增长率、近三年净利润复合增长率、股息率、PEG。

In [0]:
roe20.data.update(roe20.pool.track)
roe20.value.update(roe20.pool.track,[5,3,1],force=True)
roe20.change.update(roe20.pool.track,['close','pe','pb'],force=True)

## 3.估值表
- PE-TTM：滚动市盈率，为近五年的数据；
- 高度：表示时间窗内（五年），低于当前值的个数与全部样本个数的比例，代表当前值在历史统计值的估值位置；
- 最小值：表示时间窗内（五年）理论最小估值，表示其估值下限；
- 最大值：表示时间窗内（五年）理论最大估值，表示其估值上限；
- 中位数：表示时间窗内（五年），代表该指数的理论合理估值；
- 距底：表示当前值与最低值的距离，代表最大的可能跌幅。 

In [0]:
view.table()

## 4.行情

In [14]:
view.change()

../PF\pf.py:1252: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  p = ax.imshow(df, cmap=plt.cm.jet, interpolation='nearest')


ValueError: 'box_aspect' and 'fig_aspect' must be positive

## 5.走势图

In [0]:
view.line_one()

In [0]:
view.line_compar()